In [ ]:
%matplotlib inline
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib.dates import DateFormatter, date2num
import datetime as dt
import matplotlib.ticker as ticker
import geopandas
from shapely.geometry import shape

# Load data from nytimes
# Has data for each county in each state
df = pd.read_csv('https://raw.githubusercontent.com/nytimes/covid-19-data/master/us-counties.csv', 
                 parse_dates=['date'])


JSON_url = "https://raw.githubusercontent.com/deldersveld/topojson/master/countries/us-states"
SC_name = "South Carolina"
SC_json = "SC-45-south-carolina-counties"
SC_abr = "SC"


In [ ]:
# Print the data
df.head()

In [ ]:
# Select South Carolina
state = df[df.state == SC_name]
# Group by county for each date
state = state.pivot(index='date', columns='county', values='cases')
state = state.fillna(0.0)
state.tail(5)

In [ ]:
plot = state.plot(figsize=(12,8), linewidth=5, cmap='PuBu')
plot.legend(ncol=2, bbox_to_anchor=(1, 1), loc='upper left')
plt.show()

In [ ]:
# Show plotting inside of pandas
plot = state.Richland.plot(figsize=(12,8), linewidth=5, legend=True)
plot = state.Lexington.plot(linewidth=5, legend=True)

In [ ]:
# Show plotting inside of pandas (log this time)
plot = state.Richland.plot(figsize=(12,8), linewidth=5, legend=True)
plot = state.Lexington.plot(linewidth=5, legend=True)
plot.set_yscale('log')

In [ ]:
# Show how to plot dates with matplotlib plot_date

fig, ax = plt.subplots(figsize=(16,9))

days_to_average = 5

dates = date2num(state.index)
ax.plot_date(dates, state.Richland.rolling(days_to_average).mean()*days_to_average, fmt='-', linewidth=5)
ax.plot_date(dates, state.Lexington.rolling(days_to_average).mean()*days_to_average, fmt='-', linewidth=5)
date_form = DateFormatter("%b-%d")
ax.xaxis.set_major_formatter(date_form)
ax.set_yscale('log')
plt.show()

In [ ]:
# Copy data
new_per_day = state.copy(deep=True)

# Get difference per day instead of total cases
for county in state.columns:
    new_per_day[county] = new_per_day[county].diff()
new_per_day = new_per_day.fillna(0.0)
new_per_day.tail(5) 


In [ ]:
# Plot difference
# Looks jumpy....
plot = new_per_day.Richland.plot(figsize=(12,8), linewidth=5, legend=True)

#plot.set_yscale('log')


In [ ]:
# Plot again using matplotib
fig, ax = plt.subplots(figsize=(16,9))

# Average over the last two weeks
rolling_days = 7

dates = date2num(state.index)
npd_richland_avg = new_per_day.Richland.rolling(rolling_days).mean()

# This time we do rolling mean to take some of the jittering out
ax.plot_date(dates, npd_richland_avg, fmt='-', linewidth=5)

# Plot trend line for the last set of days averaged over
z = np.polyfit(dates[-rolling_days:], npd_richland_avg[-rolling_days:], 1)
p = np.poly1d(z)
ax.plot(dates[-rolling_days:],p(dates[-rolling_days:]),'--', linewidth=3, label=f'Trend for last {rolling_days} days')
ax.bar(dates, new_per_day.Richland, alpha=0.2)

date_form = DateFormatter("%b-%d")
ax.xaxis.set_major_formatter(date_form)
#ax.set_yscale('log')
ax.legend()
plt.show()

In [ ]:
# Get geometry data
# https://github.com/deldersveld/topojson
geodata = geopandas.read_file(JSON_url+"/"+SC_json+".json")

In [ ]:
fig, ax = plt.subplots(figsize=(16,9))
geodata.plot(color="white", edgecolor='black', ax=ax)
max_count = np.max(state.max(axis=0))

geodata["color"] = np.zeros_like(geodata.NAME)

for c in state.iteritems():
    geodata.loc[geodata.NAME == c[0], 'color'] = c[1][-1]/max_count

geodata.plot(column='color', ax=ax, cmap='PuBu')
ax.set_title("Current Cases in SC")
ax.set_yticklabels([])
ax.set_xticklabels([])
ax.get_yaxis().set_visible(False)
ax.get_xaxis().set_visible(False)
plt.show()
plt.close()